In [1]:
#paquetes python
import os
import numpy as np
import copy
import pprint
from math import sqrt
from scipy.linalg import solve_triangular
import inspect
import sys

#paquetes funciones definidas obtenidas de archivos .py
import funciones_factorizacion_QR as fQR

**OBJETIVO:**

El presente notebook tiene por finalidad exhibir y poner a prueba funciones definidas por el equipo QR para resolver un Sistema de Ecuaciones Lineales (SEL), a través de 3 pasos:

    1) Obtención de factorización QR (A=QR)
    2) Implementación del método de eliminación por bloques
    3) Resolución de SEL (Ax=b) con los 2 pasos anteriores
    
Para lograr tal objetivo se presentarán a continuacion las funciones implementadas en python, así como una serie de ejemplos para entender mejor su funcionamiento

## Factorización QR

### Funciones

In [2]:
lines=inspect.getsource(fQR.crear_matriz_aleatoria)
print(lines)

def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias

    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstanc

In [3]:
help(fQR.crear_matriz_aleatoria)

Help on function crear_matriz_aleatoria in module funciones_factorizacion_QR:

crear_matriz_aleatoria(renglones, columnas, maximo_valor, minimo_valor, entero=False)
    
    Descripción:
            Función que genera matrices de tamaño definido con entradas dentro del rango
            indicado
    
    Parametros:
            renglones       integer indica renglones de la matriz
            columnas        integer indica columnas de la matriz
            maximo_valor    integer o double señala valor máximo de las entradas de la matriz
            minimo_valor    integer o double señala valor mínimo de las entradas de la matriz
            entero          Boolean, indica si las entradas serán enteras (True) o no (False)
    
    Resultado:
            M               np.ndarray que representa matriz (mxn) con numeros al azar en el
                            rango indicado
    
    Ejemplos:
    Crear matriz de m=4 renglones y n=3 columnas, con entradas reales entre 3.4 y 15.6:
    cr

In [4]:
lines=inspect.getsource(fQR.house)
print(lines)

def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
     

In [5]:
help(fQR.house)

Help on function house in module funciones_factorizacion_QR:

house(x)
    
    Descripción:
            Función que calcula vector de householder
    
    Parametros:
            x       np.ndarray que representa vector al que se le hará la reflexión householder
    
    Resultado:
            Beta    double para obtener matriz de reflexión householder v
            v       np.ndarray que representa vector que representa la reflexión de householder
    
    Con el vector v y el factor beta obtenidos se puede calcular la matriz de reflexión de
    householder R, donde R= Identidad(mxm) - Beta*v*v_transpuesta (v*v_transpuesta es matriz  
    de pxp si v tiene p entradas). El vector v está construido de tal forma que al generar la 
    matriz de reflexión de householder Rf y multiplicarse por x se obtiene una proyección de 
    este último sobre el vector canónico e_1=(1,0,0,...,0) obteniendo un vector con todas sus 
    entradas ingualas  a cero excepto la primera, que de hecho es ||x||

In [6]:
lines=inspect.getsource(fQR.matriz_auxiliar_Arv)
print(lines)

def matriz_auxiliar_Arv(A):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    m,n=A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    
    #m contiene el numero de renglones y n el de columnas
    m=A.shape[0]


In [7]:
help(fQR.matriz_auxiliar_Arv)

Help on function matriz_auxiliar_Arv in module funciones_factorizacion_QR:

matriz_auxiliar_Arv(A)
    
    Descripción:
            Función que genera una matriz que contiene los elementos r distintos de cero de 
            la matriz R y las entradas de los vectores householder v (excepto la primera), 
            con los cuales se puede calcular la matriz Q. Ambas matrices componentes de la 
            factorización QR
    
    Parametros:
            A      numpy.ndarray que representa matriz (mxn) de la que se desea obtener 
                   elemento de factorización QR
    
    Resultado:
            Arv    numpy.ndarray que representa matriz (mxn) que incluye las componentes 
                   distintas de cero de la matriz R y los vectores householder con los 
                   que se puede obtener la matriz Q, y con ello la factorización QR
    
    La factorización QR consiste en encontrar 2 matrices tales que A=Q*R 
    donde Q es una matriz ortogonal y R es una matriz 

In [8]:
lines=inspect.getsource(fQR.matriz_Q_R)
print(lines)

def matriz_Q_R(A):
    """
    Función que devuelve la matriz R y Q de la factorización QR de una matriz A
    
    params: A    Matriz (mxn)

    return: Q    Matriz Q (mxm) de la factorización A=QR
            R    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    elif A.shape[0]<A.shape[1]:
        sys.exit('El numero de renglones de A tiene que ser igual o mayor al no. de columnas')
    
    Arv=matriz_auxiliar_Arv(A)
    m,n=A.shape
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(n):
        Qj=Q_j(Arv,j+1)
        Q=Q@Qj
        R=Q_j(Arv,j+1)@R
    return Q,R



In [9]:
help(fQR.matriz_Q_R)

Help on function matriz_Q_R in module funciones_factorizacion_QR:

matriz_Q_R(A)
    
    Descripción:
            Función que devuelve la matriz Q y R de la factorización QR de una matriz A
    
    Parametros:
            A     np.ndarray que representa una matriz (mxn)
    
    Resultado:
            R     np.ndarray que representa matriz R (mxm) de la factorización A=QR
            Q     np.ndarray que representa matriz Q (mxm) de la factorización A=QR
    
    
    Para obtener la factorización primero se calcula la matriz auxiliar Arv. La matriz Q 
    (matriz ortogonal tal que Q_transpuesta*Q es igual a la identidad (mxm)
    se calcula como el producto de las matrices Q_j's, las cuales se obtienen a partir
    de las diferentes matrices de reflexión householder utilizadas para generar los ceros
    en las columnas de A para formar la matriz R
    
    Q = Q_1*Q_2*Q_3*Q_4
    
    Nota: Cada matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo 
    vector de house

In [10]:
lines=inspect.getsource(fQR.Q_j)
print(lines)

def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j

    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m,n=Arv.shape
    Qj=np.eye(m)
    #Para construir Q

In [11]:
help(fQR.Q_j)

Help on function Q_j in module funciones_factorizacion_QR:

Q_j(Arv, j)
    
    Descripción:
            Función que calcula la matriz Qj (en el proceso de obtención de 
            factorización QR se van obteniendo n Qj's, que si se multiplican todas 
            da por resultado Q=Q_1*Q_2*...*Q_n)
    
    Parámetros:                            
            A_r_v   np.ndarray que representa la matriz auxiliar (mxn) que incluye 
                elementos de la matriz R y Q, de la factorización QR
            j       integer que indica el índice de la matriz Q_j
    
    Resultado:
            Qj      np.ndarray que representa la j-esima transformación de householder
    
    La matriz Q_j se calcula con el vector v_j, correspondiente al j-ésimo vector 
    de householder como: Q_j = Identidad(mxm) - Beta*v_j*v_j_transpuesta



## Factorización QR

### Ejemplos

In [12]:
#Pra probar el funcionamiento de estas funciones puede definirse una matriz entrada por entrada
#o usar la función crear_matriz_aleatoria

#Ejemplo a mano (asegurandose de que sus entradas sean tipo dobles -aunque sólo se capture
#una parte entera-, pues de lo contrario se tendrán errores por redondeo significativos)
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [13]:
x=np.array([1,2,3.])
print("x=\n",x)

Beta,v=fQR.house(x)
print("\n\nBeta=\n",np.round(Beta,4))
print("\n\nv=\n",np.round(v,4))

#Calculando matriz de reflexión de householder R
R=np.eye(3)-Beta*np.outer(v,v)
print("\n\nR=\n",np.round(R,4))

#Observese como el producto de Rx da por resultado un vector con
#todas sus entradas igualas a cero excepto la primera
print("\n\nRx=\n",np.round(R@x,4))

x=
 [1. 2. 3.]


Beta=
 0.7327


v=
 [ 1.     -0.7295 -1.0942]


R=
 [[ 0.2673  0.5345  0.8018]
 [ 0.5345  0.6101 -0.5849]
 [ 0.8018 -0.5849  0.1227]]


Rx=
 [3.7417 0.     0.    ]


In [14]:
#Definase una matriz A
A=fQR.crear_matriz_aleatoria(6,4,6,-6)
print('\n\nA=\n',np.round(A,4))

#Calculamos la matriz auxiliar Arv para visualizarla
Arv=fQR.matriz_auxiliar_Arv(A)
print('\n\nArv=\n',np.round(Arv,4))

#Obtenemos la matriz Q y R a partir de Arv
Q,R=fQR.matriz_Q_R(A)
print('\n\nQ=\n',np.round(Q,4))
print('\n\nR=\n',np.round(R,4))

#Checamos que QR=A
print('\n\nQ@R=\n',np.round(Q@R,4))
print('\n\nA=\n',np.round(A,4))



A=
 [[-4.0569 -3.6384 -2.9198 -2.663 ]
 [-1.754   3.5447 -4.2457  4.928 ]
 [-5.116   2.0583  1.8531 -4.7973]
 [ 0.9473  1.8715 -3.0936  1.4429]
 [-4.5414 -4.273   4.4925  3.6795]
 [-4.0209  2.4589 -0.7115 -2.0647]]


Arv=
 [[ 9.1323  1.0188 -1.1673  2.1529]
 [ 0.133   7.5371 -3.4706 -0.3993]
 [ 0.3879 -1.1458  6.8437 -1.0776]
 [-0.0718 -0.4557  0.7029  8.2654]
 [ 0.3443  0.7914 -1.1202 -0.7796]
 [ 0.3049 -1.1499  0.1623 -0.686 ]]


Q=
 [[-0.4442 -0.4227 -0.7168 -0.3203 -0.0265 -0.0833]
 [-0.1921  0.4963 -0.4015  0.6179 -0.245  -0.3375]
 [-0.5602  0.3488  0.3521 -0.3717  0.2817 -0.4722]
 [ 0.1037  0.2343 -0.3155  0.1177  0.8729  0.2428]
 [-0.4973 -0.4997  0.3182  0.592   0.1978  0.1101]
 [-0.4403  0.3858  0.0166 -0.1143 -0.2426  0.7649]]


R=
 [[ 9.1323  1.0188 -1.1673  2.1529]
 [-0.      7.5371 -3.4706 -0.3993]
 [-0.     -0.      6.8437 -1.0776]
 [-0.     -0.     -0.      8.2654]
 [-0.      0.      0.     -0.    ]
 [-0.      0.      0.      0.    ]]


Q@R=
 [[-4.0569 -3.6384 -2.9198 

In [15]:
#Comparemos con el código de numpy
Q_np, R_np=np.linalg.qr(A)
#notamos que numpy obtiene con signos contrarios Q y R
print('Q_np=\n',np.round(Q_np,4))
#Además R no guarda toda la matriz R de mxn, sino sólo la parte superior de nxn
print('\nR_np=\n',np.round(R_np,4))
#la multiplicación da por resultado A
print('\nQ_np@R_np=\n',np.round(Q_np@R_np,4))

Q_np=
 [[-0.4442  0.4227  0.7168 -0.3203]
 [-0.1921 -0.4963  0.4015  0.6179]
 [-0.5602 -0.3488 -0.3521 -0.3717]
 [ 0.1037 -0.2343  0.3155  0.1177]
 [-0.4973  0.4997 -0.3182  0.592 ]
 [-0.4403 -0.3858 -0.0166 -0.1143]]

R_np=
 [[ 9.1323  1.0188 -1.1673  2.1529]
 [ 0.     -7.5371  3.4706  0.3993]
 [ 0.      0.     -6.8437  1.0776]
 [ 0.      0.      0.      8.2654]]

Q_np@R_np=
 [[-4.0569 -3.6384 -2.9198 -2.663 ]
 [-1.754   3.5447 -4.2457  4.928 ]
 [-5.116   2.0583  1.8531 -4.7973]
 [ 0.9473  1.8715 -3.0936  1.4429]
 [-4.5414 -4.273   4.4925  3.6795]
 [-4.0209  2.4589 -0.7115 -2.0647]]


In [16]:
#Observese que Q=(Q_4*Q_3*Q_2*Q_1)_transpuesta
np.round(np.transpose(fQR.Q_j(Arv,4)@fQR.Q_j(Arv,3)@fQR.Q_j(Arv,2)@fQR.Q_j(Arv,1)),4)

array([[-0.4442, -0.4227, -0.7168, -0.3203, -0.0265, -0.0833],
       [-0.1921,  0.4963, -0.4015,  0.6179, -0.245 , -0.3375],
       [-0.5602,  0.3488,  0.3521, -0.3717,  0.2817, -0.4722],
       [ 0.1037,  0.2343, -0.3155,  0.1177,  0.8729,  0.2428],
       [-0.4973, -0.4997,  0.3182,  0.592 ,  0.1978,  0.1101],
       [-0.4403,  0.3858,  0.0166, -0.1143, -0.2426,  0.7649]])

In [17]:
#Puede comprobarse que Q_transpuesta*Q es igual a la identidad
print(np.round(np.transpose(Q)@Q,4))

[[ 1. -0. -0. -0.  0.  0.]
 [-0.  1.  0. -0.  0. -0.]
 [-0.  0.  1. -0.  0.  0.]
 [-0. -0. -0.  1. -0. -0.]
 [ 0.  0.  0. -0.  1. -0.]
 [ 0. -0.  0. -0. -0.  1.]]


In [18]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
A=fQR.crear_matriz_aleatoria(5,5,5,-5,True)
print('\n\nA=\n',np.round(A,4))

m=A.shape[0]
n=A.shape[1]
Q=np.eye(m)
R=copy.copy(A)
Arv=fQR.matriz_auxiliar_Arv(A)
for j in range(n):
    if j==0:
        cadena='Q'+str(j+1)
    else:
        cadena='Q'+str(j+1) + ' @ ' + cadena
    Qj=fQR.Q_j(Arv,j+1)
    print('\nQ',j+1,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=fQR.Q_j(Arv,j+1)@R
    print('\n',cadena,'@ A=\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))
print('\nQ@R:\n',np.round(Q@R,4))
print('\nA:\n',np.round(A,4))



A=
 [[-2. -4. -4.  4. -1.]
 [ 2.  1.  5. -4.  5.]
 [ 1.  2.  1. -5.  5.]
 [-2. -5. -1. -3.  3.]
 [ 1. -4.  4.  4.  2.]]

Q 1 :
 [[-0.5345  0.5345  0.2673 -0.5345  0.2673]
 [ 0.5345  0.8138 -0.0931  0.1862 -0.0931]
 [ 0.2673 -0.0931  0.9535  0.0931 -0.0465]
 [-0.5345  0.1862  0.0931  0.8138  0.0931]
 [ 0.2673 -0.0931 -0.0465  0.0931  0.9535]]

 Q1 @ A=
 [[ 3.7417  4.8107  6.6815 -2.9399  3.4744]
 [-0.     -2.069   1.2793 -1.5826  3.4414]
 [-0.      0.4655 -0.8604 -3.7913  4.2207]
 [ 0.     -1.931   2.7207 -5.4174  4.5586]
 [-0.     -5.5345  2.1396  5.2087  1.2207]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.     -0.3319  0.0747 -0.3098 -0.8879]
 [ 0.      0.0747  0.9958  0.0174  0.0498]
 [ 0.     -0.3098  0.0174  0.928  -0.2065]
 [ 0.     -0.8879  0.0498 -0.2065  0.4082]]

 Q2 @ Q1 @ A=
 [[ 3.7417  4.8107  6.6815 -2.9399  3.4744]
 [ 0.      6.2335 -3.2314 -2.7043 -3.323 ]
 [-0.      0.     -0.6075 -3.7284  4.6   ]
 [ 0.     -0.      1.6717 -5.6782  2.9853]
 [ 0.     -0.  

In [19]:
#Se prueba un ejemplo más grande la función matriz_auxiliar_Arv
#que es base para la obtención de Q y R
A=fQR.crear_matriz_aleatoria(11**2,10**2,10,-10,False)
Arv=fQR.matriz_auxiliar_Arv(A)
Arv

array([[ 6.39245028e+01,  2.82438413e+00, -2.70735278e+00, ...,
         4.35546276e+00, -7.65059694e+00,  7.14299663e+00],
       [-1.39267488e-01,  6.40847817e+01, -2.72243815e+00, ...,
        -2.71358615e+00,  4.68095550e+00,  9.28885845e+00],
       [ 9.38451166e-03, -1.21785917e-02,  6.41358414e+01, ...,
         4.11821613e+00,  8.43966758e-01,  3.01693834e+00],
       ...,
       [ 8.57949383e-02,  1.04559400e-01, -1.12798431e-01, ...,
         1.07045792e-01,  4.89674648e-02, -1.69449972e-01],
       [-7.45331911e-02,  1.26698115e-01, -7.96881896e-02, ...,
         8.75364198e-03, -5.97565735e-02, -3.94687602e-01],
       [-1.09114859e-01,  8.85415624e-02,  1.28277690e-01, ...,
         1.10165663e-01, -1.53976073e-01, -4.08919242e-01]])

## Solución de Sistemas de Ecuaciones Lineales

### Funciones

In [20]:
lines=inspect.getsource(fQR.Solucion_SEL_QR_nxn)
print(lines)

def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
    #elif b.shape[0]!= m or b.shape[1]!= 1:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("

In [21]:
help(fQR.Solucion_SEL_QR_nxn)

Help on function Solucion_SEL_QR_nxn in module funciones_factorizacion_QR:

Solucion_SEL_QR_nxn(A, b)
    
    Descripción:
            Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con 
        n ecuaciones y n incognitas
    
    Parámetros:                            
            A      Matriz (nxn) que representa los coeficientas de las ecuaciones 
               elementos de la matriz R y Q, de la factorización QR
            b      vector (nx1) constantes del sistema
    
    Resultado:
            x      vector que satisface (Ax=b)
    
    El Sistema de Ecuaciones Lineales establecido como Ax=b, es resuelto con esta función
    mediante la factorización QR como a continuación se explica:
    
      Si A*x = b y A = Q*R, con A matriz (mxn), b (mx1) y Q una matriz ortogonal (mxm) y 
                                R (mxn) una matriz triangular superior
      entonces Q*R*x = b.
      
      Aprovechando la propiedad de Q^(-1)=Q, es decir, Q^(-1)*Q=Ide

## Eliminación por bloques

### Funciones

In [22]:
lines=inspect.getsource(fQR.crear_bloques)
print(lines)

def crear_bloques(A, b, m1=False, n1=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            m1  Numero de columnas que tendrá el 1er bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A deb

In [23]:
help(fQR.crear_bloques)

Help on function crear_bloques in module funciones_factorizacion_QR:

crear_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que divide la matriz A(mxn) y el vector b(mx1) en cuatro submatrices y dos vectores respectivamente
    
    Parámetros:                            
      A     Matriz (mxn) 
      b     vector (mx1)
      m1    Número de renglones del los bloques superiores
      n1    Número de columnas de los bloques de la izquierda
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
      b es un parámetro opcional, si no es introducido solo regresa las submatrices de A
    
    
    Resultado:
      A11   numpy.ndarray que representa la matriz de dimensión m1xn1
      A12   numpy.ndarray que representa la matriz de dimensión m2xn1
      A12   numpy.ndarray que representa la matriz de dimensión m1xn2
      A22   numpy.ndarray que representa la matriz 

In [24]:
lines=inspect.getsource(fQR.eliminacion_bloques)
print(lines)

def eliminacion_bloques(A,b, m1=False, n1=False):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
        
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:


In [25]:
help(fQR.eliminacion_bloques)

Help on function eliminacion_bloques in module funciones_factorizacion_QR:

eliminacion_bloques(A, b, m1=False, n1=False)
    
    Descripción:
      Función que usa la eliminación por bloques para resolver un sistema de ecuaciones Ax=b.
    
    Parámetros:                            
      A     numpy.ndarray que representa la matriz (mxn) que representan loc coeficientes del sistema de      ecuaciones
      b     numpy.ndarray que representa el vector (mx1) que representan el lado derecho del sistema de       ecuaciones
      m1    Número de renglones del los bloques superiores del subconjunto a resolver
      n1    Número de columnas de los bloques del subconjunto a resolver
    
      En el caso en que no se de m1 o n1, éstos parámetros serán igual a la parte entera del número de renglones o el número de columnas entre dos.
    
    
    Resultado:
      x   numpy.ndarray que representa la solución al sistema de ecuaciones Ax=b
    
    Este método consiste en eliminar un subconju

## Solución de Sistemas de Ecuaciones Lineales

### Ejemplos

### A) Sistema con soluciones únicas

In [26]:
#Generamos un sistema de ecuaciones lineales
m=12
n=12
A=np.round(fQR.crear_matriz_aleatoria(m,n,6,-6,False),2)
b = np.round(fQR.crear_matriz_aleatoria(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-1.88  0.36 -2.22  2.76  2.41  3.89  1.82 -1.82 -3.59 -2.39 -2.26 -1.55]
 [-3.51  1.56  1.37  3.24 -2.43  3.16 -4.85  1.91 -3.39 -2.24 -2.54  4.34]
 [-0.3   3.07 -5.07 -3.98  0.71  3.62  0.19  0.69 -1.63  0.47  2.39 -0.67]
 [ 1.14  2.07 -0.5  -3.24  3.26 -4.15 -5.91 -3.93  5.43  3.64 -5.93 -3.04]
 [-0.53  2.48  3.9   3.38  5.25  5.69 -3.82 -1.77 -2.04 -3.86 -3.87 -1.77]
 [ 4.65  4.74 -3.56  4.16  2.04  1.21 -3.9   1.6   5.88  2.01  1.75 -1.12]
 [-3.8   1.54 -1.17  1.54  5.31 -2.38  1.98  5.43 -2.43  3.46 -5.4  -1.88]
 [-3.19 -2.18 -4.33  1.99  0.81  1.58 -2.43  0.29  0.75 -0.04  3.11  3.22]
 [ 3.94 -3.28  4.72 -1.2   0.26 -0.16  3.66 -2.94  4.23 -3.48  2.9  -5.78]
 [-0.11  5.57  2.82 -3.08  1.18  4.16  3.62  4.37  0.91  1.33  1.58 -5.09]
 [ 2.04  0.59  4.43 -1.74  1.36 -0.75  1.12  5.98  5.99  4.19  0.4  -5.24]
 [ 0.4   2.25  2.41 -0.05  3.9   4.62 -3.88  0.83 -4.86  2.68  3.19 -3.35]]
b: [[-3.47]
 [ 4.93]
 [-3.53]
 [ 1.75]
 [-1.21]
 [-2.33]
 [-0.31]
 [-3.97]
 [-5.2 ]
 [ 5.01]
 [ 

Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [27]:
np.linalg.solve(A,b)

array([[-0.54950782],
       [ 0.70981782],
       [ 1.00459932],
       [ 0.19777804],
       [-0.84407887],
       [ 0.38607155],
       [ 0.27650284],
       [-0.50449669],
       [ 0.15322839],
       [ 1.38554301],
       [-0.25355661],
       [ 0.43533786]])

Ahora usemos la factorización QR

In [28]:
x=fQR.Solucion_SEL_QR_nxn(A,b)
x

array([[-0.54950782],
       [ 0.70981782],
       [ 1.00459932],
       [ 0.19777804],
       [-0.84407887],
       [ 0.38607155],
       [ 0.27650284],
       [-0.50449669],
       [ 0.15322839],
       [ 1.38554301],
       [-0.25355661],
       [ 0.43533786]])

Por último usemos eliminación por bloques con QR

In [29]:
fQR.eliminacion_bloques(A,b)

array([[-0.54950782],
       [ 0.70981782],
       [ 1.00459932],
       [ 0.19777804],
       [-0.84407887],
       [ 0.38607155],
       [ 0.27650284],
       [-0.50449669],
       [ 0.15322839],
       [ 1.38554301],
       [-0.25355661],
       [ 0.43533786]])

### B) Sistema sin solución

In [34]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [5]], dtype='d')
print("A:",A)
print("b:",b)

A: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
b: [[3.]
 [3.]
 [5.]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [35]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

* Marca un error: "Singular matrix"

Ahora usemos la eliminación por bloques usando la factorización QR

In [41]:
fQR.eliminacion_bloques(A,b)

SystemExit: A debe ser no singular

* Se concluye que es un sistema de rectas paralelas, por lo tanto no existe solúción.

### C) Sistema con infinitas soluciones

In [43]:
#Generamos un sistema de ecuaciones lineales
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]], dtype='d')
b = np.array([[3], [3], [0]], dtype='d')
print("A:",A)
print("b:",b)

A: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
b: [[3.]
 [3.]
 [0.]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [45]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

* Marca un error: "Singular matrix"

Ahora usemos eliminación por bloques usando la factorización QR

In [46]:
fQR.eliminacion_bloques(A,b)

SystemExit: A debe ser no singular